In [1]:
import sys, time, argparse
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.contrib.layers import l2_regularizer
from tensorflow.contrib.layers import batch_norm

_VALIDATION_RATIO = 0.1

In [2]:
data = np.load("SAH_binary.matrix", allow_pickle=True)
data = np.clip(data, 0, 1)
cov = np.load("SAH_binary.covs", allow_pickle=True)

In [3]:
cov

[[0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0

In [4]:
data.shape

(65988, 200)

In [5]:
inputDim=data.shape[1]
covDim=len(cov[0])

In [6]:
embeddingDim=128
randomDim=128
generatorDims=(128, 128)
discriminatorDims=(256, 128, 1)
compressDims=()
decompressDims=()
bnDecay=0.99
l2scale=0.001

In [7]:
generatorDims = list(generatorDims) + [embeddingDim]
print(generatorDims)

[128, 128, 128]


In [8]:
aeActivation = tf.nn.tanh
generatorActivation = tf.nn.relu
discriminatorActivation = tf.nn.relu
compressDims = list(compressDims) + [embeddingDim]
decompressDims = list(decompressDims) + [inputDim]
print(compressDims,decompressDims)

[128] [200]


In [9]:
x_raw = tf.placeholder('float', [None, inputDim])
x_random= tf.placeholder('float', [None, randomDim])
x_cov = tf.placeholder('float', [None, covDim])
keep_prob = tf.placeholder('float')
bn_train = tf.placeholder('bool')
print(x_raw,"/",x_random)
print(keep_prob,"/",bn_train)

Tensor("Placeholder:0", shape=(?, 200), dtype=float32) / Tensor("Placeholder_1:0", shape=(?, 128), dtype=float32)
Tensor("Placeholder_3:0", dtype=float32) / Tensor("Placeholder_4:0", dtype=bool)


In [10]:
###apply the autoencoder to learn the salient features
def buildAutoencoder(x_input, x_cov):
        decodeVariables = {}
        with tf.variable_scope('autoencoder', regularizer=l2_regularizer(l2scale)):
            tempVec = x_input
            tempDim = inputDim
            ad_tempVec = tf.concat([tempVec, x_cov], 1) #[?,203]
            ad_tempDim = tempDim + covDim #203
  
            i = 0
            for compressDim in compressDims:
                W = tf.get_variable('aee_W_'+str(i), shape=[ad_tempDim, compressDim]) #[203,128]
                b = tf.get_variable('aee_b_'+str(i), shape=[compressDim])  #[128]
                ad_tempVec = aeActivation(tf.add(tf.matmul(ad_tempVec, W), b))  #[?,128]
                ad_tempDim = compressDim  #128
                i += 1
                
            i = 0
            for decompressDim in decompressDims[:-1]:
                ##add cov for decompress too
                ad_tempVec = tf.concat([ad_tempVec, x_cov], 1)
                ad_tempDim = ad_tempDim + covDim
                
                W = tf.get_variable('aed_W_'+str(i), shape=[ad_tempDim, decompressDim])
                b = tf.get_variable('aed_b_'+str(i), shape=[decompressDim])
                ad_tempVec = aeActivation(tf.add(tf.matmul(ad_tempVec, W), b))
                ad_tempDim = decompressDim
                decodeVariables['aed_W_'+str(i)] = W
                decodeVariables['aed_b_'+str(i)] = b
                i += 1
                
            ##add cov for decompress too
            ad_tempVec = tf.concat([ad_tempVec, x_cov], 1)
            ad_tempDim = ad_tempDim + covDim
            
            W = tf.get_variable('aed_W_'+str(i), shape=[ad_tempDim, decompressDims[-1]])
            b = tf.get_variable('aed_b_'+str(i), shape=[decompressDims[-1]])
            decodeVariables['aed_W_'+str(i)] = W
            decodeVariables['aed_b_'+str(i)] = b

            x_reconst = tf.nn.sigmoid(tf.add(tf.matmul(ad_tempVec,W),b))
            loss = tf.reduce_mean(-tf.reduce_sum(x_input * tf.log(x_reconst + 1e-12) + (1. - x_input) * tf.log(1. - x_reconst + 1e-12), 1), 0)
           
        return loss, decodeVariables

In [ ]:
loss_ae, decodeVariables = buildAutoencoder(x_raw, x_cov)
print(loss_ae,"|",decodeVariables)

In [11]:
def buildGenerator(x_input,x_cov, bn_train):
        tempVec = x_input
        tempDim = randomDim
        ad_tempVec = tf.concat([tempVec, x_cov], 1)
        ad_tempDim = tempDim + covDim
        with tf.variable_scope('generator', regularizer=l2_regularizer(l2scale)):
            for i, genDim in enumerate(generatorDims[:-1]):
                W = tf.get_variable('W_'+str(i), shape=[ad_tempDim, genDim])
                h = tf.matmul(ad_tempVec,W)
                h2 = batch_norm(h, decay=bnDecay, scale=True, is_training=bn_train, updates_collections=None)
                h3 = generatorActivation(h2)
                tempVec = h3 + tempVec
                tempDim = genDim
            W = tf.get_variable('W'+str(i), shape=[ad_tempDim, generatorDims[-1]])
            h = tf.matmul(ad_tempVec,W)
            h2 = batch_norm(h, decay=bnDecay, scale=True, is_training=bn_train, updates_collections=None)

            h3 = tf.nn.tanh(h2)
            
            output = h3 + tempVec
        return output

In [ ]:
x_fake = buildGenerator(x_random, x_cov, bn_train)
print(x_fake)

In [12]:
def getDiscriminatorResults(x_input, x_cov, keepRate, reuse=False):
        batchSize = tf.shape(x_input)[0]
        inputMean = tf.reshape(tf.tile(tf.reduce_mean(x_input,0), [batchSize]), (batchSize, inputDim))
        tempVec = tf.concat([x_input, inputMean], 1)
        tempDim = inputDim * 2
        ad_tempVec = tf.concat([tempVec, x_cov], 1)
        ad_tempDim = tempDim + covDim        
        with tf.variable_scope('discriminator', reuse=reuse, regularizer=l2_regularizer(l2scale)):
            for i, discDim in enumerate(discriminatorDims[:-1]):
                W = tf.get_variable('W_'+str(i), shape=[ad_tempDim, discDim])
                b = tf.get_variable('b_'+str(i), shape=[discDim])
                h = discriminatorActivation(tf.add(tf.matmul(ad_tempVec,W),b))
                h = tf.nn.dropout(h, keepRate)
                ad_tempVec = h
                ad_tempDim = discDim
            W = tf.get_variable('W', shape=[ad_tempDim, 1])
            b = tf.get_variable('b', shape=[1])
            y_hat = tf.squeeze(tf.nn.sigmoid(tf.add(tf.matmul(ad_tempVec, W), b)))
        return y_hat

In [13]:
def buildDiscriminator(x_real, x_fake, x_cov, keepRate, decodeVariables, bn_train):
        #Discriminate for real samples
        y_hat_real = getDiscriminatorResults(x_real, x_cov, keepRate, reuse=False)

        #Decompress, then discriminate for real samples
        tempVec = x_fake
        tempVec = tf.concat([tempVec, x_cov], 1)
        i = 0
        for _ in decompressDims[:-1]:
            tempVec = aeActivation(tf.add(tf.matmul(tempVec, decodeVariables['aed_W_'+str(i)]), decodeVariables['aed_b_'+str(i)]))
            i += 1

        x_decoded = tf.nn.sigmoid(tf.add(tf.matmul(tempVec, decodeVariables['aed_W_'+str(i)]), decodeVariables['aed_b_'+str(i)]))
        
        y_hat_fake = getDiscriminatorResults(x_decoded, x_cov, keepRate, reuse=True)

        loss_d = -tf.reduce_mean(tf.log(y_hat_real + 1e-12)) - tf.reduce_mean(tf.log(1. - y_hat_fake + 1e-12))
        loss_g = -tf.reduce_mean(tf.log(y_hat_fake + 1e-12))

        return loss_d, loss_g, y_hat_real, y_hat_fake

In [ ]:
loss_d, loss_g, y_hat_real, y_hat_fake = buildDiscriminator(x_raw, x_fake, x_cov, keep_prob, decodeVariables, bn_train)
print(loss_d,"/",loss_g)
print(y_hat_real,"/", y_hat_fake)

In [ ]:
covArray = np.asarray(cov)

In [ ]:
covArray=covArray.astype('float32')

In [ ]:
newMat = np.concatenate((data,covArray),axis=1)

In [ ]:
newMat.dtype

In [ ]:
newMat.shape

In [ ]:
train, valid = train_test_split(newMat, test_size=_VALIDATION_RATIO, random_state=0)

In [ ]:
train.shape

In [ ]:
valid.shape

In [ ]:
trainX, trainCov = train[:, :200], train[:,200:203]

In [ ]:
trainX.shape

In [ ]:
trainCov.shape

In [ ]:
validX, validCov = valid[:, :200], valid[:,200:203]

In [ ]:
validX.shape

In [ ]:
validCov.shape

In [ ]:
t_vars = tf.trainable_variables()
ae_vars = [var for var in t_vars if 'autoencoder' in var.name]
d_vars = [var for var in t_vars if 'discriminator' in var.name]
g_vars = [var for var in t_vars if 'generator' in var.name]

In [ ]:
all_regs = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
optimize_ae = tf.train.AdamOptimizer().minimize(loss_ae + sum(all_regs), var_list=ae_vars)
optimize_d = tf.train.AdamOptimizer().minimize(loss_d + sum(all_regs), var_list=d_vars)
decodeVariablesValues = list(decodeVariables.values())
optimize_g = tf.train.AdamOptimizer().minimize(loss_g + sum(all_regs), var_list=g_vars+decodeVariablesValues)

In [ ]:
batchSize=1
saveMaxKeep=0
initOp = tf.global_variables_initializer()
nBatches = int(np.ceil(float(trainX.shape[0]) / float(batchSize)))
saver = tf.train.Saver(max_to_keep=saveMaxKeep)
modelPath=''
pretrainEpochs=1

In [ ]:
a, b = buildAutoencoder(trainX, trainCov)

In [ ]:
randomX = np.random.normal(size=(batchSize, randomDim))

In [ ]:
randomX = randomX.astype('float32')

In [ ]:
index = np.random.choice(covArray.shape[0], batchSize, replace=False)  

In [ ]:
randomCov = covArray[index]

In [ ]:
c = buildGenerator(randomX, randomCov, bn_train)
print(c)

In [ ]:
pretrainBatchSize=1
modelPath=""
with tf.Session() as sess:
            if modelPath == '': sess.run(initOp)
            else: saver.restore(sess, modelPath)
            nTrainBatches = int(np.ceil(float(trainX.shape[0])) / float(pretrainBatchSize))
            nValidBatches = int(np.ceil(float(validX.shape[0])) / float(pretrainBatchSize))
            
            if modelPath== '':
                for epoch in range(pretrainEpochs):
                    idx = np.random.permutation(trainX.shape[0])
                    trainLossVec = []
                    for i in range(nTrainBatches):
                        batchX = trainX[idx[i*pretrainBatchSize:(i+1)*pretrainBatchSize]]
                        _, batchCov = trainCov[idx[i*pretrainBatchSize:(i+1)*pretrainBatchSize]]
                        _,loss = sess.run([optimize_ae, loss_ae], feed_dict={x_raw:batchX,x_cov:batchCov})
                        trainLossVec.append(loss)
                    idx = np.random.permutation(validX.shape[0])
                    validLossVec = []
                    for i in range(nValidBatches):
                        batchX = validX[idx[i*pretrainBatchSize:(i+1)*pretrainBatchSize]]
                        batchCov = validCov[idx[i*pretrainBatchSize:(i+1)*pretrainBatchSize]] 
                        loss = sess.run(loss_ae, feed_dict={x_raw:batchX,x_cov:batchCov})
                        validLossVec.append(loss)
                    validReverseLoss = 0.
                    buf = 'Pretrain_Epoch:%d, trainLoss:%f, validLoss:%f, validReverseLoss:%f' % (epoch, np.mean(trainLossVec), np.mean(validLossVec), validReverseLoss)
                    print(buf)